In [1]:
import itertools
import nltk
import jieba
vocabulary_size = 800
unknown_token = u"U"
sentence_start_token = u"S"
sentence_end_token = u"E"
%load_ext autoreload
%autoreload 2

jieba 分词

In [94]:
print "Read txt file..."
with open("data/ppd/input.txt","rb") as f:
    reader = f.readlines()
    sentences = [list(jieba.cut(x)) for x in reader if len(x) > 3]
#     sentences=["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in reader]

Read txt file...


In [95]:
print len(sentences)

3842


In [53]:
raw= sentences[:]

In [96]:
# add S E
for a in sentences:
    a.insert(0,sentence_start_token)
    a.insert(len(a)-1,sentence_end_token)

In [97]:
for a in sentences[50:60]:
    for w in a:
        print w,

S 只好 装 得 放荡 E 
S 飞来飞去 E 
S 我 飞来飞去 E 
S 满怀希望 E 
S 我 像 一只 小鸟 E 
S 我 感觉 不到 倦意 E 
S 却 又 无处可去 E 
S 空虚 把 我 仍 在 街上 E 
S 像 个 病人 逃避 死亡 E 
S 这里 适合 游荡 E 


In [67]:
# tostring
for a in sentences:
    for w in a:
        print w,

S 我 真的 需要 你 E 
S 现在 我 觉得 有些 孤单 E 
S 悲哀 的 自我 有些 辛酸 E 
S 没有 爱 也 没有 存款 E 
S 只有 去 幻想 才能 感到 一丝 温暖 E 
S 每天 早晨 我数 一二三 E 
S 爬 起床 来 看见 阳光灿烂 E 
S 大街 上 落叶 纷纷 E 
S 商店 里 放 着 怀旧 歌曲 E 
S 我 真的 真的 需要 E 
S 我 真的 需要 你 E 
S 当 夜晚 降临 繁星 满天 E 
S 我 灵魂 的 影子 靠 在 那个 墙上 E 
S 没有 脸 也 没有 心脏 E 
S 在 长安街 上 像 朵 苍白 的 花 E 
S 我 倾听 着 静脉 里 血 的 流淌 E 
S 就 像 那 昨夜 漫长 冷漠 的 细雨 E 
S 我 睁 着眼 许多 门 在 面前 紧闭 E 
S 现在 我 真的 那么 真的 E 
S 我 真的 我 真的 需要 E 
S 我 真的 需要 你 E 
S 每天 我 疲惫 地 回到 家里 E 
S 躺 在 床上 听 着 收音机 里 的 浪漫 E 
S 回忆 着 过去 的 幸福 E 
S 呢喃 着 现实 的 渺茫   E 
S 爱情 是 放在 兜里 的 一颗 炸弹 E 
S 生活 像件 背心 破烂不堪 E 
S 现在 我 不再 需要 啤酒 和 上帝 E  


In [98]:
print type(sentences)
print type(sentences[0])
print type(sentences[1])

<type 'list'>
<type 'list'>
<type 'list'>


In [61]:
# already is list.type ,do not use word_tokenize
tokenized_sentences = [nltk.word_tokenize(sent.decode('utf-8')) for sent in sentences]

AttributeError: 'list' object has no attribute 'decode'

In [16]:
for a in tokenized_sentences[0]:print a,

S 我 真 的 需 要 你 E


In [100]:
for a in sentences[100]:print a,

S 他 有 一个 美满 的 好 生活 E 


In [101]:
word_freq = nltk.FreqDist(itertools.chain(*sentences))

In [102]:
for i in word_freq.keys()[10:20]:
    print i,word_freq[i]
len(word_freq)

坚定 1
喝完 1
认不出 2
不治之症 1
乐园 1
这条 6
什么样 4
般 9
发生 7
如同 7


3283

In [118]:
vocab = word_freq.most_common(3283)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [119]:
for i in xrange(20,30):
    print i,index_to_word[i]


20 感觉
21 都
22 知道
23 生命
24 不
25 爱
26 自己
27 有
28 地
29 会


In [105]:
for i, sent in enumerate(sentences):
    sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [27]:
print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]


Example sentence: 'S 我 真 的 需 要 你
 E'

Example sentence after Pre-processing: '[u'S', u'\u6211', u'\u771f', u'\u7684', u'\u9700', u'\u8981', u'\u4f60', u'E']'


In [120]:
import numpy as np
X_train = np.asarray([ np.asarray([word_to_index[w] for w in sent[:-1]]) for sent in sentences])
y_train = np.asarray([ np.asarray([word_to_index[w] for w in sent[1:]]) for sent in sentences])

In [121]:
print X_train.shape
print y_train.shape

(3842L,)
(3842L,)


In [34]:
print index_to_word[141]

E


In [110]:
for a in sentences[1000]:print a,
print X_train[1000]
print y_train[1000]

S 我 是 你 弱小 的 孩子 E 
[  1   4  14   5 733   3 103   0]
[  4  14   5 733   3 103   0   2]


In [127]:
from rnnlm import RNNLM
hdim = 100
np.random.seed(10)
L0 = np.zeros((3284,hdim))
model = RNNLM(L0,U0=L0,alpha=0.01,rseed=10,bptt=3)
# model.grad_check(np.array([1,2,3]),np.array([2,3,4]))

In [128]:
from rnnlm import RNNLM
epoch = 4
nn = len(y_train)
X = X_train[:nn]
Y = y_train[:nn]
idxiter = np.random.permutation(range(nn)*epoch)
print "Using %d train example" % nn

model.train_sgd(X,Y,idxiter=idxiter)

Using 3842 train example
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.13835
  Seen 10000 in 312.42 s
  [10000]: mean loss 4.60573
  [15368]: mean loss 4.53843
SGD complete: 15368 examples in 514.04 seconds.


[(0, 8.1383533367346921),
 (10000, 4.6057307216439938),
 (15368, 4.5384317953327509)]

In [145]:
import cPickle as pickle
def save_(name,p):
    with open(name,"w") as f:pickle.dump(p, f)

In [146]:
save_("wf.L.npy", model.sparams.L)
save_("wf.U.npy", model.params.U)
save_("wf.H.npy", model.params.H)

In [134]:
def seq_to_words(seq):
    return [index_to_word[s] for s in seq]
    
seq, J = model.generate_sequence(word_to_index[sentence_start_token], 
                                 word_to_index[sentence_end_token], 
                                 maxlen=80)
print J
# print seq
print " ".join(seq_to_words(seq))

27.985226324
S 我 的 感觉 紧紧 永生 E


In [137]:
print " ".join(seq_to_words(model.generate_sequence(word_to_index[sentence_start_token], 
                                 word_to_index[sentence_end_token], 
                                 maxlen=80)[0]))

S 辉煌 那 现实   到底 带你去 恋人 E


In [138]:
fg = lambda :" ".join(seq_to_words(model.generate_sequence(word_to_index[sentence_start_token], 
                                 word_to_index[sentence_end_token], 
                                 maxlen=80)[0]))

In [148]:
for i in xrange(10):
    print fg()

S 我 的 好点 再也 了 E
S 眩晕 你 没有 会 地方 杀人 E
S 我 夜里 你 怎样 我 低保金 更 蓝天 走来 的 街道 如同 E
S 就 相信 青春 幸福 E
S 我 再见 有意思 你 像 幻觉 早点 E
S 轻轻地 无论 U 的 我们 E
S 是 那 我 奇幻 青稞 我 一种 窒息 E
S 我 明天 一 的 孩子 神往 不想 E
S 幻觉 你 见 最后 我 这里 E
S 车里 一天 我 不再 的 了 公主 E
